In [16]:
import pandas as pd

In [17]:
%run "../read_and_write_docs.py"

In [18]:
base_loc = "../../../../datasets/PAN/PAN 21/"
unknown_train_loc = f"{base_loc}pan21-train-y.jsonl"
unknown_test_loc = f"{base_loc}pan21-test-y.jsonl"
potential_impostor_loc = f"{base_loc}pan21-potential-impostors.jsonl"
train_impostors_loc = f"{base_loc}pan21-train-impostors.jsonl"
test_impostors_loc = f"{base_loc}pan21-test-impostors.jsonl"

In [19]:
train = read_jsonl_file(unknown_train_loc)
test = read_jsonl_file(unknown_test_loc)
potential_impostors = read_jsonl_file(potential_impostor_loc)

In [21]:
def create_samples(train, potential_impostors, n):
    results = []

    for sample_id in train['sample_id'].unique():
        sample_row = train[train['sample_id'] == sample_id].iloc[0]
        current_author = sample_row['author']
        
        # Exclude the current author from potential impostors
        available_impostors = potential_impostors[potential_impostors['author'] != current_author].copy()
        
        for _ in range(n):
            if available_impostors.empty:
                break
                
            # Sample one row from available impostors
            sampled_impostor = available_impostors.sample(n=1)
            
            # Add the result to the list
            results.append({
                'sample_id': sample_id,
                'author': sampled_impostor['author'].values[0],
                'fandom': sampled_impostor['fandom'].values[0],
                'text': sampled_impostor['text'].values[0]
            })
            
            # Exclude this author from future sampling
            sampled_author = sampled_impostor['author'].values[0]
            available_impostors = available_impostors[available_impostors['author'] != sampled_author]
    
    return pd.DataFrame(results)

In [22]:
train_imp = create_samples(train, potential_impostors, 50)
test_imp = create_samples(test, potential_impostors, 50)

In [23]:
save_as_jsonl(train_imp, train_impostors_loc)
save_as_jsonl(test_imp, test_impostors_loc)